In [15]:
import json
from dotenv import load_dotenv 
import time

load_dotenv()

In [2]:
!pip show openai | grep Version

Version: 1.3.5


In [4]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [10]:
from openai import OpenAI

client = OpenAI()

## Create the OpenAI Assistant

In [11]:
assistantone = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistantone)

{'id': 'asst_bZNdoacKlcfEQy1lk58C1Lyk',
 'created_at': 1701088677,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

## Create an instance of an OpenAI tread for a single user interaction

In [12]:
threadone = client.beta.threads.create()
show_json(threadone)

{'id': 'thread_nEUYAGd8qBQNhyysyeH36325',
 'created_at': 1701088809,
 'metadata': {},
 'object': 'thread'}

## Add a new message from the user to ThreadOne

In [13]:
message = client.beta.threads.messages.create(
    thread_id=threadone.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_XhAZJd7dv4fYTOerdIfP0gpL',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1701088941,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_nEUYAGd8qBQNhyysyeH36325'}

## Create a Run to have the AssistantOne respond to the message in ThreadOne

In [14]:
runone = client.beta.threads.runs.create(
    thread_id=threadone.id,
    assistant_id=assistantone.id,
)
show_json(runone)

{'id': 'run_Ob8wHDmfkuHbHPj5j3Mthp9j',
 'assistant_id': 'asst_bZNdoacKlcfEQy1lk58C1Lyk',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1701089071,
 'expires_at': 1701089671,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_nEUYAGd8qBQNhyysyeH36325',
 'tools': []}

## Poll the Status of the Run which is Async

In [16]:
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [17]:
runone = wait_on_run(runone, threadone)
show_json(runone)

{'id': 'run_Ob8wHDmfkuHbHPj5j3Mthp9j',
 'assistant_id': 'asst_bZNdoacKlcfEQy1lk58C1Lyk',
 'cancelled_at': None,
 'completed_at': 1701089074,
 'created_at': 1701089071,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1701089072,
 'status': 'completed',
 'thread_id': 'thread_nEUYAGd8qBQNhyysyeH36325',
 'tools': []}

## Return the message from the Assistant to the users Question

In [18]:
curr_onemessages = client.beta.threads.messages.list(thread_id=threadone.id)
show_json(curr_onemessages)

{'data': [{'id': 'msg_1CyplGdwE8k7GCHpubMJI2zS',
   'assistant_id': 'asst_bZNdoacKlcfEQy1lk58C1Lyk',
   'content': [{'text': {'annotations': [],
      'value': 'Yes. To solve for x, subtract 11 from both sides to get `3x = 3`, and then divide both sides by 3 to find `x = 1`.'},
     'type': 'text'}],
   'created_at': 1701089073,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_Ob8wHDmfkuHbHPj5j3Mthp9j',
   'thread_id': 'thread_nEUYAGd8qBQNhyysyeH36325'},
  {'id': 'msg_XhAZJd7dv4fYTOerdIfP0gpL',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1701088941,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_nEUYAGd8qBQNhyysyeH36325'}],
 'object': 'list',
 'first_id': 'msg_1CyplGdwE8k7GCHpubMJI2zS',
 'last_id': 